In [1]:
# from matplotlib import pyplot as plt

# H = nx.gaussian_random_partition_graph(n=100,s=10,v=10,p_in=.25,p_out=.1)
# H = nx.relaxed_caveman_graph(l=3,k=15,p=0.15)
# H= nx.powerlaw_cluster_graph(n=200,m=10,p=0.2)
# H = nx.geographical_threshold_graph(200, 60)

# nx.draw(H)
# plt.figure(2,figsize=(20,20))
# nx.draw_spring(H,node_size=100,font_size=8) 
# plt.show()

In [2]:
# get the files from model.py
import math
from enum import Enum
import networkx as nx

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid

# Model

In [3]:
#### THIS CAN BE SUBSTITUTED WITH BIAS ##########
#Define the state of an individual
class State(Enum):
    SUSCEPTIBLE = 0
    INFECTED = 1
    RESISTANT = 2

In [4]:
def number_state(model, state):
    """
    GET THE TOTAL NUMBER OF PEOPLE FOR THE STATE YOU ARE LOOKING FOR
    
    sudo code
    for person in model.grid.get_all_cell_contents():
        if person.state is state:
            add 1 to the sum
    """
    return sum([1 for a in model.grid.get_all_cell_contents() if a.state is state])


def number_infected(model):
    """
    use the number_state function to get back the total number of INFECTED people
    """
    return number_state(model, State.INFECTED)


def number_susceptible(model):
    """
    use the number_state function to get back the total number of SUSCEPTIBLE people
    """
    return number_state(model, State.SUSCEPTIBLE)


def number_resistant(model):
    """
    use the number_state function to get back the total number of RESISTANT people
    """
    return number_state(model, State.RESISTANT)

In [5]:
class VirusOnNetwork(Model):
    """A virus model with some number of agents"""
    
    """MODEL CONSTRUCTOR NEEDS AN __init__ method and step method"""
    def __init__(
        self,
        num_nodes=10,
        avg_node_degree=3,
        initial_outbreak_size=1,
        virus_spread_chance=0.4,
        virus_check_frequency=0.4,
        recovery_chance=0.3,
        gain_resistance_chance=0.5,
    ):

        #INITIALIZE THE NETWORK, CHANGE HERE TO OTHER GRAPH TYPES
        self.num_nodes = num_nodes
        prob = avg_node_degree / self.num_nodes
        self.G = nx.erdos_renyi_graph(n=self.num_nodes, p=prob)
        
        #initialize the rest of the model attributes
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        self.initial_outbreak_size = (
            initial_outbreak_size if initial_outbreak_size <= num_nodes else num_nodes
        )
        self.virus_spread_chance = virus_spread_chance
        self.virus_check_frequency = virus_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

        #initialize the datacollector
        self.datacollector = DataCollector(
            {
                "Infected": number_infected,
                "Susceptible": number_susceptible,
                "Resistant": number_resistant,
            }
        )
        ##END CONSTRUCTOR

        # HERE WE BUILD THE NETWORK
        # Create agents
        for i, node in enumerate(self.G.nodes()):
            #make an agent
            a = VirusAgent(
                i,
                self,
                State.SUSCEPTIBLE, # HERE WE CAN RANDOMLY PICK BIAS
                self.virus_spread_chance,
                self.virus_check_frequency,
                self.recovery_chance,
                self.gain_resistance_chance,
            )
            self.schedule.add(a)
            # Add the agent to the node
            self.grid.place_agent(a, node)

        # Infect some nodes
        infected_nodes = self.random.sample(self.G.nodes(), self.initial_outbreak_size)
        for a in self.grid.get_cell_list_contents(infected_nodes):
            a.state = State.INFECTED

        self.running = True
        self.datacollector.collect(self)

    def resistant_susceptible_ratio(self):
        try:
            return number_state(self, State.RESISTANT) / number_state(
                self, State.SUSCEPTIBLE
            )
        except ZeroDivisionError:
            return math.inf

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in range(n):
            self.step()

In [6]:
class VirusAgent(Agent):
    # THIS COULD BE THE USER
    
    #CONSTRUCTOR -- REPLACE WITH USER ATTRIBUTES
    def __init__(
        self,
        unique_id,
        model,
        initial_state,
        virus_spread_chance,
        virus_check_frequency,
        recovery_chance,
        gain_resistance_chance,
    ):
        super().__init__(unique_id, model)

        self.state = initial_state

        self.virus_spread_chance = virus_spread_chance
        self.virus_check_frequency = virus_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

    def try_to_infect_neighbors(self):
        # THE WHOLE THING IS RANDOM
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_neighbors = [
            agent
            for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            if agent.state is State.SUSCEPTIBLE
        ]
        for a in susceptible_neighbors:
            if self.random.random() < self.virus_spread_chance:
                a.state = State.INFECTED

    def try_gain_resistance(self):
        # THE WHOLE THING IS RANDOM
        if self.random.random() < self.gain_resistance_chance:
            self.state = State.RESISTANT

    def try_remove_infection(self):
        # Try to remove
        if self.random.random() < self.recovery_chance:
            # Success
            self.state = State.SUSCEPTIBLE
            self.try_gain_resistance()
        else:
            # Failed
            self.state = State.INFECTED

    def try_check_situation(self):
        if self.random.random() < self.virus_check_frequency:
            # Checking...
            if self.state is State.INFECTED:
                self.try_remove_infection()

    def step(self):
        if self.state is State.INFECTED:
            self.try_to_infect_neighbors()
        self.try_check_situation()

# Server

In [7]:
# get the files from server.py
import math

from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.modules import TextElement
#from .model import VirusOnNetwork, State, number_infected

In [8]:
def network_portrayal(G):
    # The model ensures there is always 1 agent per node

    def node_color(agent):
        return {State.INFECTED: "#FF0000", State.SUSCEPTIBLE: "#008000"}.get(
            agent.state, "#808080"
        )

    def edge_color(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return "#000000"
        return "#e8e8e8"

    def edge_width(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return 3
        return 2

    def get_agents(source, target):
        return G.nodes[source]["agent"][0], G.nodes[target]["agent"][0]

    portrayal = dict()
    portrayal["nodes"] = [
        {
            "size": 6,
            "color": node_color(agents[0]),
            "tooltip": "id: {}<br>state: {}".format(
                agents[0].unique_id, agents[0].state.name
            ),
        }
        for (_, agents) in G.nodes.data("agent")
    ]

    portrayal["edges"] = [
        {
            "source": source,
            "target": target,
            "color": edge_color(*get_agents(source, target)),
            "width": edge_width(*get_agents(source, target)),
        }
        for (source, target) in G.edges
    ]

    return portrayal


network = NetworkModule(network_portrayal, 500, 500, library="d3")
chart = ChartModule(
    [
        {"Label": "Infected", "Color": "#FF0000"},
        {"Label": "Susceptible", "Color": "#008000"},
        {"Label": "Resistant", "Color": "#808080"},
    ]
)


class MyTextElement(TextElement):
    def render(self, model):
        ratio = model.resistant_susceptible_ratio()
        ratio_text = "&infin;" if ratio is math.inf else "{0:.2f}".format(ratio)
        infected_text = str(number_infected(model))

        return "Resistant/Susceptible Ratio: {}<br>Infected Remaining: {}".format(
            ratio_text, infected_text
        )


model_params = {
    "num_nodes": UserSettableParameter(
        "slider",
        "Number of agents",
        10,
        10,
        100,
        1,
        description="Choose how many agents to include in the model",
    ),
    "avg_node_degree": UserSettableParameter(
        "slider", "Avg Node Degree", 3, 3, 8, 1, description="Avg Node Degree"
    ),
    "initial_outbreak_size": UserSettableParameter(
        "slider",
        "Initial Outbreak Size",
        1,
        1,
        10,
        1,
        description="Initial Outbreak Size",
    ),
    "virus_spread_chance": UserSettableParameter(
        "slider",
        "Virus Spread Chance",
        0.4,
        0.0,
        1.0,
        0.1,
        description="Probability that susceptible neighbor will be infected",
    ),
    "virus_check_frequency": UserSettableParameter(
        "slider",
        "Virus Check Frequency",
        0.4,
        0.0,
        1.0,
        0.1,
        description="Frequency the nodes check whether they are infected by " "a virus",
    ),
    "recovery_chance": UserSettableParameter(
        "slider",
        "Recovery Chance",
        0.3,
        0.0,
        1.0,
        0.1,
        description="Probability that the virus will be removed",
    ),
    "gain_resistance_chance": UserSettableParameter(
        "slider",
        "Gain Resistance Chance",
        0.5,
        0.0,
        1.0,
        0.1,
        description="Probability that a recovered agent will become "
        "resistant to this virus in the future",
    ),
}

server = ModularServer(
    VirusOnNetwork, [network, MyTextElement(), chart], "Virus Model", model_params
)
server.port = 8521

In [9]:
server.launch()

Interface starting at http://127.0.0.1:8521
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"submit_params","param":"num_nodes","value":11}
{"type":"submit_params","param":"num_nodes","value":12}
{"type":"submit_params","param":"num_nodes","value":13}
{"type":"submit_params","param":"num_nodes","value":15}
{"type":"submit_params","param":"num_nodes","value":17}
{"type":"submit_params","param":"num_nodes","value":20}
{"type":"submit_params","param":"num_nodes","value":21}
{"type":"submit_params","param":"num_nodes","value":22}
{"type":"submit_params","param":"num_nodes","value":23}
{"type":"submit_params","param":"num_nodes","value":25}
{"type":"submit_params","param":"num_nodes","value":27}
{"type":"submit_params","param":"num_nodes","value":28}
{"type":"submit_params","param":"num_nodes","value":30}
{"type":"submit_params","param":"num_nodes","value":31}
{"type":"submit_params","param":"num_nodes","value":33}
{"type":"submit_params","param":"num_nodes","value":35}
{"type